# Clustering

In [640]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
from mpl_toolkits import mplot3d
from sklearn.cluster import KMeans
from sklearn.cluster import DBSCAN
import math

In [641]:
#load dataset

#Einlesen der Daten
df = pd.read_csv(r"konto_32197.CSV", sep =";")
print(df)

     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    07.01.2021   
1    32197  Sonstige Verbindlichkeiten 04     62586257    20.12.2021   
2    32197  Sonstige Verbindlichkeiten 04     74258581    02.12.2021   
3    32197  Sonstige Verbindlichkeiten 04     78813645    23.12.2021   
4    32197  Sonstige Verbindlichkeiten 04     77284095    27.04.2021   
..     ...                            ...          ...           ...   
225  32197  Sonstige Verbindlichkeiten 04     77294541    22.12.2021   
226  32197  Sonstige Verbindlichkeiten 04     70269632    22.12.2021   
227  32197  Sonstige Verbindlichkeiten 04     63997907    23.12.2021   
228  32197  Sonstige Verbindlichkeiten 04     63724817    23.12.2021   
229  32197  Sonstige Verbindlichkeiten 04     79238525           NaN   

     Betrag EUR  
0      3.887,18  
1          7,68  
2       -955,63  
3        -36,02  
4     59.472,09  
..          ...  
225     -

In [642]:
#preprocessing
print(df)
def preprocessing(df):
    # Delete useless Columns
    #df.drop('Konto', axis=1, inplace=True)
    #df.drop('Kontoname', axis=1, inplace=True)    
    #df.dropna(axis='index', how='all', inplace=True)
    df = df.dropna()
    df['Buchungsdatum'] = pd.to_datetime(df['Buchungsdatum'], format='%d.%m.%Y')

    # Create absolute Column
    for idx, row in df.iterrows():
        try:
            df.at[idx, 'Betrag EUR'] = float(row['Betrag EUR'])
        except ValueError:
            pass
    df['Betrag EUR'] = df['Betrag EUR'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float).round(2)
    df['Absolut'] = df['Betrag EUR'].abs()
    
    #df['Absolute'] = df['Betrag EUR'].apply(abs)
    df['Binary sign'] = df['Betrag EUR'].apply(lambda x: 1 if x>0 else -1)
    return df

df = preprocessing(df)
print(df)

#berechne die Summe der offenen Posten
sum = df["Betrag EUR"].sum()
sum = round(sum, 2)
print(sum) 

     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    07.01.2021   
1    32197  Sonstige Verbindlichkeiten 04     62586257    20.12.2021   
2    32197  Sonstige Verbindlichkeiten 04     74258581    02.12.2021   
3    32197  Sonstige Verbindlichkeiten 04     78813645    23.12.2021   
4    32197  Sonstige Verbindlichkeiten 04     77284095    27.04.2021   
..     ...                            ...          ...           ...   
225  32197  Sonstige Verbindlichkeiten 04     77294541    22.12.2021   
226  32197  Sonstige Verbindlichkeiten 04     70269632    22.12.2021   
227  32197  Sonstige Verbindlichkeiten 04     63997907    23.12.2021   
228  32197  Sonstige Verbindlichkeiten 04     63724817    23.12.2021   
229  32197  Sonstige Verbindlichkeiten 04     79238525           NaN   

     Betrag EUR  
0      3.887,18  
1          7,68  
2       -955,63  
3        -36,02  
4     59.472,09  
..          ...  
225     -

C:\Users\canis\AppData\Local\Temp\ipykernel_12544\870747538.py:9: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Buchungsdatum'] = pd.to_datetime(df['Buchungsdatum'], format='%d.%m.%Y')
C:\Users\canis\AppData\Local\Temp\ipykernel_12544\870747538.py:17: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  df['Betrag EUR'] = df['Betrag EUR'].str.replace('.', '', regex=True).str.replace(',', '.').astype(float).round(2)
C:\Users\canis\AppData\Local\Temp\ipykernel_12544\870747538.py:18: SettingWithCopyWarning: 
A va

In [643]:
# group by absolute value (create transaction pairs)
grouped_transactions = df.groupby('Betrag EUR')

# Accessing groups (clusters) of transactions with equal absolute amounts
equal_abs_amount_clusters = [group for _, group in grouped_transactions]

# Print the clusters
for i, cluster in enumerate(equal_abs_amount_clusters):
    print(f'Cluster {i+1}:')
    print(cluster)
    print('\n')

Cluster 1:
    Konto                      Kontoname  Belegnummer Buchungsdatum  \
20  32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   

    Betrag EUR   Absolut  Binary sign  
20   -98690.94  98690.94           -1  


Cluster 2:
   Konto                      Kontoname  Belegnummer Buchungsdatum  \
5  32197  Sonstige Verbindlichkeiten 04     64725542    2021-04-27   

   Betrag EUR   Absolut  Binary sign  
5   -58555.65  58555.65           -1  


Cluster 3:
    Konto                      Kontoname  Belegnummer Buchungsdatum  \
98  32197  Sonstige Verbindlichkeiten 04     70444210    2021-05-20   

    Betrag EUR   Absolut  Binary sign  
98   -21237.65  21237.65           -1  


Cluster 4:
     Konto                      Kontoname  Belegnummer Buchungsdatum  \
194  32197  Sonstige Verbindlichkeiten 04     64992163    2021-11-04   

     Betrag EUR  Absolut  Binary sign  
194    -20707.6  20707.6           -1  


Cluster 5:
    Konto                      Kontoname  Beleg




Cluster 42:
    Konto                      Kontoname  Belegnummer Buchungsdatum  \
49  32197  Sonstige Verbindlichkeiten 04     65336679    2021-02-19   

    Betrag EUR   Absolut  Binary sign  
49   -13752.03  13752.03           -1  


Cluster 43:
    Konto                      Kontoname  Belegnummer Buchungsdatum  \
22  32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   

    Betrag EUR   Absolut  Binary sign  
22   -13741.67  13741.67           -1  


Cluster 44:
     Konto                      Kontoname  Belegnummer Buchungsdatum  \
132  32197  Sonstige Verbindlichkeiten 04     77833209    2021-07-15   

     Betrag EUR   Absolut  Binary sign  
132   -13142.81  13142.81           -1  


Cluster 45:
     Konto                      Kontoname  Belegnummer Buchungsdatum  \
122  32197  Sonstige Verbindlichkeiten 04     61866765    2021-06-24   

     Betrag EUR   Absolut  Binary sign  
122   -12924.72  12924.72           -1  


Cluster 46:
     Konto                   

In [644]:
# Sort data by 'Buchungsdatum' to ensure it's in chronological order
df = df.sort_values(by='Buchungsdatum')
print(df)

     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign  
0       3887.18   3887.18            1  
23       -16.67     16.67           -1  
22    -13741

In [645]:
#erstelle ids für später für jeden eintraG in df
df['Id'] = range(len(df))
print(df)

     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign   Id  
0       3887.18   3887.18            1    0  
23       -16.67     16.67           -1    1

In [646]:
#gruppieren nach datum über ids
def group_by_date(df):
    """
    Diese Funktion gruppiert ein DataFrame nach dem Datum und gibt die IDs als Liste für jedes Datum zurück.
    
    Args:
        df (pandas.DataFrame): Das Eingabe-DataFrame.
        
    Returns:
        pandas.DataFrame: Ein DataFrame mit zwei Spalten: 'Buchungsdatum' und 'IDs'.
    """
    # Gruppiere nach Datum und sammle die IDs in einer Liste
    df_grouped_by_date = df.groupby('Buchungsdatum')['Id'].apply(list).reset_index()
    df_grouped_by_date.columns = ['Buchungsdatum', 'IDs']
    
    return df_grouped_by_date

df_grouped_by_date = group_by_date(df)

print(df_grouped_by_date)

    Buchungsdatum              IDs
0      2021-01-07     [0, 1, 2, 3]
1      2021-01-08              [4]
2      2021-01-14              [5]
3      2021-01-15           [6, 7]
4      2021-01-18              [8]
..            ...              ...
116    2021-12-20  [218, 219, 220]
117    2021-12-22       [221, 222]
118    2021-12-23  [223, 224, 225]
119    2021-12-27            [226]
120    2021-12-28       [227, 228]

[121 rows x 2 columns]


In [647]:
# def find_and_remove_linear_combinations(df):
#     # Iteriere über die Zeilen
#     for i, row in df.iterrows():
#         ids = row['IDs']
        
#         # Erstelle eine Teilmenge des DataFrames mit den ausgewählten IDs
#         subset = df[df['IDs'].apply(lambda x: all(id in x for id in ids))]
        
#         # Berechne die Summe der Beträge
#         total_amount = subset['Betrag EUR'].sum()
        
#         # Überprüfe, ob die Summe der Beträge gleich Null ist
#         if round(total_amount, 2) == 0.0:
#             # Falls ja, entferne die IDs aus dem ursprünglichen DataFrame
#             df.at[i, 'IDs'] = []
    
#     return df

# print(df)

     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign   Id  
0       3887.18   3887.18            1    0  
23       -16.67     16.67           -1    1

In [648]:
def preprocess_and_group(df):
    # Create absolute column
    df['Betrag EUR'] = df['Betrag EUR'].str.replace('.', '', regex=True).astype(float).round(2)
    df['Absolut'] = df['Betrag EUR'].abs()
    df['Binary sign'] = df['Betrag EUR'].apply(lambda x: 1 if x > 0 else 0)

    # Group by 'Buchungsdatum' and calculate sums of positive and negative entries
    grouped_df = df.groupby(['Buchungsdatum', 'Binary sign'])['Absolut'].sum().reset_index()

    # Filter out entries that cancel each other out
    grouped_df = grouped_df[grouped_df['Absolut'] != 0]

    return grouped_df

In [649]:
#bilde die summe über jedes datum (gruppiere über die Beträge indem du die jeweiligen werte für die zeilen über die ids raussuchst)
# Erstelle eine leere Liste, um die Summen zu speichern
sum_values = []

# Gehe durch jede Zeile im DataFrame 'df_grouped_by_date'
for index, row in df_grouped_by_date.iterrows():
    # Hole die IDs aus der Spalte 'IDs' für diese Zeile
    ids = row['IDs']
    
    # Filtere das ursprüngliche DataFrame 'df' nach den gefundenen IDs und summiere die Beträge
    sum_value = df.loc[df['Id'].isin(ids), 'Betrag EUR'].sum()
    
    # Füge die Summe zur Liste hinzu
    sum_values.append(sum_value)

# Füge die Liste der Summen als neue Spalte 'Summe' im DataFrame 'df_grouped_by_date' hinzu
df_grouped_by_date['Betrag EUR'] = sum_values

print(df_grouped_by_date)


    Buchungsdatum              IDs  Betrag EUR
0      2021-01-07     [0, 1, 2, 3]   -13758.34
1      2021-01-08              [4]    -1157.20
2      2021-01-14              [5]   -19516.46
3      2021-01-15           [6, 7]    -3367.55
4      2021-01-18              [8]      -36.86
..            ...              ...         ...
116    2021-12-20  [218, 219, 220]       -7.68
117    2021-12-22       [221, 222]   -12435.45
118    2021-12-23  [223, 224, 225]     -569.73
119    2021-12-27            [226]    98690.94
120    2021-12-28       [227, 228]     1172.88

[121 rows x 3 columns]


In [650]:
#gleitende summe erstellen
# Berechne die gleitende Summe über 30 Tage
sum_values = []

for index, row in df_grouped_by_date.iterrows():
    current_date = row['Buchungsdatum']
    start_date = current_date - pd.Timedelta(days=30)
    
    window_data = df_grouped_by_date[(df_grouped_by_date['Buchungsdatum'] >= start_date) & 
                                        (df_grouped_by_date['Buchungsdatum'] <= current_date)]
    
    sum_value = window_data['Betrag EUR'].sum()
    sum_values.append(sum_value)

df_grouped_by_date['Gleitende Summe'] = sum_values

print(df_grouped_by_date)

    Buchungsdatum              IDs  Betrag EUR  Gleitende Summe
0      2021-01-07     [0, 1, 2, 3]   -13758.34        -13758.34
1      2021-01-08              [4]    -1157.20        -14915.54
2      2021-01-14              [5]   -19516.46        -34432.00
3      2021-01-15           [6, 7]    -3367.55        -37799.55
4      2021-01-18              [8]      -36.86        -37836.41
..            ...              ...         ...              ...
116    2021-12-20  [218, 219, 220]       -7.68         14904.44
117    2021-12-22       [221, 222]   -12435.45          2468.99
118    2021-12-23  [223, 224, 225]     -569.73          1899.26
119    2021-12-27            [226]    98690.94        120379.52
120    2021-12-28       [227, 228]     1172.88        121552.40

[121 rows x 4 columns]


In [651]:
#schritt 1: lösche einträge mit gleitender summe = 0

# Annahme: df_grouped_by_date_id enthält die Daten mit Buchungsdatum, IDs und gleitender Summe

# Finde die IDs, für die die gleitende Summe gleich Null ist
zero_sum_ids = df_grouped_by_date[df_grouped_by_date['Gleitende Summe'] == 0]['IDs'].explode().tolist()

# Gib die gefundenen IDs aus
print(zero_sum_ids)

# Lösche die entsprechenden Einträge im Original-DataFrame
df = df[~df['Id'].isin(zero_sum_ids)]

# Gib die verbleibenden Einträge im originalen DataFrame aus
print(df)

# Lösche die entsprechenden IDs aus df_grouped_by_date_id
df_grouped_by_date['IDs'] = df_grouped_by_date['IDs'].apply(lambda ids: [id for id in ids if id not in zero_sum_ids])

# Gib df_grouped_by_date_id nach dem Löschen aus
print(df_grouped_by_date)

[]
     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign   Id  
0       3887.18   3887.18            1    0  
23       -16.67     16.67           -1  

In [652]:
#schritt 2: lösche einträge, bei denen sich die gleitende summe fast nich verändert

# Initialisiere eine leere Liste, um die zu löschenden IDs zu speichern
ids_to_delete = []

for i in range(len(df_grouped_by_date)-1):
    current_entry = df_grouped_by_date.iloc[i]
    next_entry = df_grouped_by_date.iloc[i+1]
    
    # Überprüfe, ob sich die gleitende Summe nicht ändert
    if current_entry['Gleitende Summe'] == next_entry['Gleitende Summe']:
        # Überprüfe, ob der Betrag EUR <= 0.1 ist
        # if round(next_entry['Betrag EUR'], 1) <= 0.1:
        #     # Füge die IDs zur Liste der zu löschenden IDs hinzu
        #     ids_to_delete.extend(next_entry['IDs'])
        # Überprüfe, ob sich die gleitende Summe nicht ändert
        ids_to_delete.extend(next_entry['IDs'])
    else:
        if math.isclose(next_entry['Betrag EUR'], 0.0, abs_tol=0.1):
                # Füge die IDs zur Liste der zu löschenden IDs hinzu
                ids_to_delete.extend(next_entry['IDs'])

# Lösche die entsprechenden Einträge im Original-DataFrame
df = df[~df['Id'].isin(ids_to_delete)]

# Lösche die IDs aus df_grouped_by_date_id
df_grouped_by_date['IDs'] = df_grouped_by_date['IDs'].apply(lambda ids: [id for id in ids if id not in ids_to_delete])

# Entferne leere Einträge aus df_grouped_by_date_id
df_grouped_by_date = df_grouped_by_date[df_grouped_by_date['IDs'].astype(bool)]

# Gib die verbleibenden Einträge im originalen DataFrame aus
print(df)
print(df_grouped_by_date)

# Zeige die Zeile für ID 227
print(df[df['Id'] == 227])

# Zeige die Zeile für ID 226
print(df[df['Id'] == 226])

# Zeige die Zeile für ID 228
print(df[df['Id'] == 228])


     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign   Id  
0       3887.18   3887.18            1    0  
23       -16.67     16.67           -1    1

In [655]:
def balance_entries(df, df_grouped_by_date):
    for i in range(len(df_grouped_by_date)-1):
        current_entry = df_grouped_by_date.iloc[i]
        next_entry = df_grouped_by_date.iloc[i+1]

        current_ids = current_entry['IDs']
        next_ids = next_entry['IDs']

        for id_1 in current_ids:
            for id_2 in next_ids:
                amount_1 = df[df['Id'] == id_1]['Betrag EUR'].values[0]
                amount_2 = df[df['Id'] == id_2]['Betrag EUR'].values[0]

                if amount_1 - amount_2 == 0.0:
                    df = df[df['Id'] != id_1]
                    df = df[df['Id'] != id_2]

                    # Entferne IDs aus df_grouped_by_date
                    df_grouped_by_date['Id'] = df_grouped_by_date['Id'].apply(lambda ids: [id for id in ids if id not in [id_1, id_2]])
    return df, df_grouped_by_date

print(df)
print(df_grouped_by_date)


     Konto                      Kontoname  Belegnummer Buchungsdatum  \
0    32197  Sonstige Verbindlichkeiten 04     76581317    2021-01-07   
23   32197  Sonstige Verbindlichkeiten 04     61897788    2021-01-07   
22   32197  Sonstige Verbindlichkeiten 04     61060446    2021-01-07   
21   32197  Sonstige Verbindlichkeiten 04     65082108    2021-01-07   
24   32197  Sonstige Verbindlichkeiten 04     72529493    2021-01-08   
..     ...                            ...          ...           ...   
227  32197  Sonstige Verbindlichkeiten 04     63997907    2021-12-23   
3    32197  Sonstige Verbindlichkeiten 04     78813645    2021-12-23   
18   32197  Sonstige Verbindlichkeiten 04     78596294    2021-12-27   
20   32197  Sonstige Verbindlichkeiten 04     78594057    2021-12-28   
19   32197  Sonstige Verbindlichkeiten 04     67418764    2021-12-28   

     Betrag EUR   Absolut  Binary sign   Id  
0       3887.18   3887.18            1    0  
23       -16.67     16.67           -1    1

In [654]:
print(len(df))

229
